In [28]:
%matplotlib inline

from __future__ import division
from pylab import *
from numpy import *
from scipy import *
import os
import scipy.io.wavfile

# Parabolic interpolation
def parabolic(f, x):
    xv = 1/2. * (f[x-1] - f[x+1]) / (f[x-1] - 2 * f[x] + f[x+1]) + x
    yv = f[x] - 1/4. * (f[x-1] - f[x+1]) * (xv - x)
    return (xv, yv)

# pobieramy liste plikow wav do wczytania
file_paths = []
for folder, subs, files in os.walk('./train'):
    for filename in files:
        file_paths.append(os.path.abspath(os.path.join(folder, filename)))
        
# Przykładowe przedziały
## korzystajac ze zbioru wartosci testowych ogarnac srednia zakres wartosci dla kobiety i mezczyzny
## wg. wiki jest to 85-180 Hz dla mezczyzny a 165-255 dla kobiety
man_freq = (80, 170)
woman_freq = (171, 255)

test_file = file_paths[1]

In [29]:
# PROBA ROZWIAZANIA ZA POMOCA TRANSFORMATY FOURIERA

def my_fft_freq(filename):
    w, signal = scipy.io.wavfile.read(filename) # problem z odczytaniem niektorych plikow

    if len(signal.shape) > 1:
        signal = [s[0] for s in signal] # niektore pliki maja dwa kanaly a inne tylko jeden - analizujemy tylko ten pierszy

    spectrum = fft(signal)

    real_values = abs(spectrum[1:len(spectrum)//2 + 1]) # pobieramy wartosci rzeczywiste z pierwszej polowy ( druga symetryczna )
    max_amplitude = max(real_values)
    index_max = np.argmax(real_values)

    freqs = np.arange(0, w/2, w / len(signal)) # tworzymy liste czestotliwosci odpowiadajaca osi X
    print(freqs[index_max]) # wybieramy z niej wartosc o najwiekszej amplitudzie i odczytujemy w ten sposob czestotliwosc tonu podstawowego 

    
my_fft_freq(test_file)

# TODO

# pliki roznia sie dlugoscia, aby przyspieszyc analize mozna analizowac tylko fragment - pytanie jak go wybrac zeby byly pelne okresy 
# jakies materialy 
# https://sound.eti.pg.gda.pl/student/akmuz/03-f0.pdf - opisana autokorelacja (nie czytalem jeszcze)
# http://stackoverflow.com/questions/30683040/matlab-how-to-find-fundamental-frequency-of-a-speech - odczytywaniu tonu podstawowego
# http://www.mathworks.com/matlabcentral/answers/28239-get-frequencies-out-of-data-with-an-fft - wzor na zamiane osi X po zrobieniu FFT na sygnale
# http://stackoverflow.com/questions/4364823/how-do-i-obtain-the-frequencies-of-each-value-in-an-fft/4371627#4371627 - podobnie ale czytelniej


487.807985818


In [86]:
# ROZWIAZANIE PRZY POMOCY AUTOKORELACJI

# wybieramy ramkę czasową od 1 do 3 sekundy

def autocorr(signal):
    original_signal = np.array(signal)
    correlations_vector = np.array([])
    for i in range(TOTAL_FRAMES):
        corelation = np.corrcoef(original_signal, signal)[0][1]
        correlations_vector = np.append(correlations_vector, corelation)
        signal = np.delete(signal, 0)
        signal = np.append(signal, 0)
        
    return correlations_vector

def find_T(filename):
    FRAME_ONE = 44100
    FRAME_TWO = 3 * 44100
    TOTAL_FRAMES =  int((FRAME_TWO - FRAME_ONE))

    w, signal = scipy.io.wavfile.read(filename)
    if len(signal.shape) > 1:
        signal = [s[0] for s in signal]

    correlations_vector = autocorr(signal)
    start = argwhere(correlations_vector <= 0)[0]
    corr = correlations_vector[start:]
    max_index = argmax(corr)
    
    print('f = ' + str(w / (start + max_index)) + ' Hz')
    
for f in file_paths[0:10]:
    try:
        if '_K' in f:
            print('Kobieta:')
        else:
            print('Mężczyzna:')

        find_T(f)
    except:
        print(f + ' file broken')
    
def autocorrelation_freq_test(filename):
    FRAME_ONE = 44100
    FRAME_TWO = 1.4 * 44100
    TOTAL_FRAMES =  int((FRAME_TWO - FRAME_ONE))

    w, signal = scipy.io.wavfile.read(filename)
    if len(signal.shape) > 1:
        signal = [s[0] for s in signal]

    correlations_vector = autocorr(signal)

    fig = plt.figure(figsize=(15,6), dpi=160)
    ax = fig.add_subplot(131)
    ax.plot(signal[int(FRAME_ONE):int(FRAME_TWO)])

    ax = fig.add_subplot(132)
    ax.plot(correlations_vector)
    
    ax = fig.add_subplot(133)
    
#     d = diff(correlations_vector)
    start = argwhere(correlations_vector <= 0)[0]
    corr = correlations_vector[start:]
    max_index = argmax(corr)
    
    print('max: ' + str(start + max_index))
    ax.plot(corr)
    
    # Find frequency
    
        
# autocorrelation_freq_test(test_file)
# find_T(file_paths[6])

# TODO -> wyciagnac maksymalna wartosc -> wyzerowac jego otoczenie (opcjonalne) -> pobrać kolejny peak
# -> obliczyc okres miedzy dwoma -> czestotliwosc = 1/okres -> uznac czy to jest K czy M
# (do okreslenia jakosci naszego algorytmu) porownac wynik dla okreslonego pliku z jego nazwa i okreslić success rate

Kobieta:


/usr/local/lib/python3.5/site-packages/scipy/io/wavfile.py:267: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


/Users/tomek/University/KCK/voice/train/001_K.wav file broken
Mężczyzna:
/Users/tomek/University/KCK/voice/train/002_M.wav file broken
Kobieta:
/Users/tomek/University/KCK/voice/train/003_K.wav file broken
Mężczyzna:
/Users/tomek/University/KCK/voice/train/004_M.wav file broken
Mężczyzna:
/Users/tomek/University/KCK/voice/train/005_M.wav file broken
Kobieta:
/Users/tomek/University/KCK/voice/train/006_K.wav file broken
Mężczyzna:
/Users/tomek/University/KCK/voice/train/007_M.wav file broken
Kobieta:


/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:27: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


f = [ 275.625] Hz
Kobieta:
f = [ 300.] Hz
Mężczyzna:
f = [ 150.] Hz


In [ ]:
# https://gist.github.com/endolith/255291#file-frequency_estimator-py-L52
from scipy.signal import fftconvolve

def find_frequency(sig, w):
    # Calculate autocorrelation (same thing as convolution, but with
    # one input reversed in time), and throw away the negative lags
    corr = fftconvolve(sig, sig[::-1], mode='full')
    corr = corr[len(corr)//2:]

    # Find the first low point
#     d = diff(corr)
    start = find(corr <= 0)[0]

    # Find the next peak after the low point (other than 0 lag).  This bit is
    # not reliable for long signals, due to the desired peak occurring between
    # samples, and other peaks appearing higher.
    # Should use a weighting function to de-emphasize the peaks at longer lags.
    peak = argmax(corr[start:])
    
    return w / (peak + start)

results = []
for f in file_paths:
    try:        
        w, signal = scipy.io.wavfile.read(f)

        gender = 'M' if '_M' in f else 'K'
        freq = find_frequency(signal, w)
        
        results.append((gender, freq))        
    except:
        print(f + " file is broken.")
        
for threshold in np.arange(100, 200):
    successes = []
    for (gender, freq) in results:
        g = 'K' if freq > threshold else 'M'

        if gender == g:
            successes.append(True)
        else:
            successes.append(False)
    
    success_rate = successes.count(True) / len(results)
    
    print('Success rate for ' + str(threshold) + ' threshold is: ' + str(success_rate) + '%')
            
        
            

/usr/local/lib/python3.5/site-packages/scipy/io/wavfile.py:267: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


/Users/tomek/University/KCK/voice/train/001_K.wav file is broken.
/Users/tomek/University/KCK/voice/train/020_M.wav file is broken.
/Users/tomek/University/KCK/voice/train/028_K.wav file is broken.
/Users/tomek/University/KCK/voice/train/029_K.wav file is broken.
/Users/tomek/University/KCK/voice/train/030_M.wav file is broken.
/Users/tomek/University/KCK/voice/train/037_K.wav file is broken.
/Users/tomek/University/KCK/voice/train/038_M.wav file is broken.
